In [1]:
import numpy as np

In [3]:
def DMD(X, Y, tol=1e-12):
    #X - Nxm, Y - Nxm
    m=X.shape[1]; N=X.shape[0]
    U, Sigma, V = np.linalg.svd(X,full_matrices=False) #svd vraca V*
    V=np.conjugate(V.T)
    #print(Sigma)
    k=m
    for i in range(1,min(N,m)):
        if(Sigma[i]<=Sigma[0]*tol):
            k=i
            break
    U = U[:, :k]; V=V[:, :k]; Sigma = np.array(Sigma[:k])
    S_k = ((np.conjugate(U.T)@Y)@V)/Sigma
    Lambda, W = np.linalg.eig(S_k)
    Z = U@W
    return Z, Lambda


In [4]:
def DMD_exact(X, Y, tol=1e-12):
    """X - Nxm, Y - Nxm; !!Exact DMD ne vraća normirane Z[:,i]"""
    m=X.shape[1]; N=X.shape[0]
    U, Sigma, V = np.linalg.svd(X,full_matrices=False) #svd vraca V*
    V=np.conjugate(V.T)
    #print(Sigma)
    k=m
    for i in range(1,min(N,m)):
        if(Sigma[i]<=Sigma[0]*tol):
            k=i
            break
    U = U[:, :k]; V=V[:, :k]; Sigma = np.array(Sigma[:k])
    S_k = ((np.conjugate(U.T)@Y)@V)/Sigma
    Lambda, W = np.linalg.eig(S_k)
    Z = (Y@V/Sigma)@W/Lambda
    return Z, Lambda

In [5]:
def GEDMD(X, Y, tol=1e-12, type: str=None): 
    """
    type = "exact" if you want exact version, or None (/leave empty) if you don't want exact version. 
    Anything else set as type will do the non-exact version.
    """
    m=X.shape[1]; N=X.shape[0]
    D = np.linalg.norm(X, axis=0)
    X = X/D; Y=Y/D
    U, Sigma, V = np.linalg.svd(X,full_matrices=False) 
    V=np.conjugate(V.T)
    k=m
    for i in range(1,min(N,m)):
        if(Sigma[i]<=Sigma[0]*tol):
            k=i
            break
    U = U[:, :k]; V=V[:, :k]; Sigma = np.array(Sigma[:k])
    B_k = Y @ (V/Sigma)
    S_k = np.conjugate(U.T)@B_k
    Lambda, W = np.linalg.eig(S_k)
    Z = U@W
    if(type=="exact"):
        Z = B_k@W
    r = B_k@W - Z*Lambda
    return Z, Lambda, r

In [ ]:
def GEDMDQ_multiple_trajectories(X, Y, tol=1e-12, type: str=None):
    """
    type = "exact" if you want exact version, or None (/leave empty) if you don't want exact version. 
    Anything else set as type will do the non-exact version.
    """
    m=X.shape[1];
    Q, R = np.linalg.qr(np.concatenate((X,Y), axis=1))
    Z, L, r = GEDMD(R[:,:m], R[:,m:], tol, type)
    Z = Q@Z
    return Z, L, r


In [ ]:
def GEDMDQ(S, tol=1e-12, type: str=None):
    """
    type = "exact" if you want exact version, or None (/leave empty) if you don't want exact version. 
    Anything else set as type will do the non-exact version.
    """
    m=S.shape[1]-1;
    Q, R = np.linalg.qr(S)
    R_x = R[:(m+1),1:m]; R_y = R[:(m+1), 2:(m+1)]
    Z, L, r = GEDMD(R_x, R_y, tol, type)
    Z = Q@Z
    return Z, L, r

In [6]:
N=300
m=10
x=np.random.rand(N)
A=np.random.rand(N,N)
A = A/np.linalg.norm(A, 2)

In [7]:
X=np.empty((N,m+1))
X[:,0]=x
for i in range(m):
    X[:,i+1]=A@X[:,i]
X=X+np.random.rand(N,m+1)/N

In [8]:
Z, L = DMD(X[:,:m], X[:, 1:], N*np.finfo(float).resolution)
#Z ima stupčanu normu otprilike 1
for i in range(Z.shape[1]):
    print(np.linalg.norm(Z[:,i]))
L

1.0
1.0
1.0
1.0
1.0
1.0000000000000002
1.0000000000000002
0.9999999999999998
1.0000000000000002
1.0000000000000002


array([ 0.99879915+0.j        ,  0.50800865+0.59611152j,
        0.50800865-0.59611152j, -0.08930496+0.74209837j,
       -0.08930496-0.74209837j,  0.06732936+0.j        ,
       -0.01976422+0.j        , -0.70964451+0.j        ,
       -0.52739543+0.49316232j, -0.52739543-0.49316232j])

In [ ]:
#permutirani podaci - druge svojstvene vrijednosti - kao što bi i očekivali!
print(X[:5,[1,6,8,9]])
temp=X[:,[1,6,8,9]]
X[:,[6,9]] = temp[:,[0,2]]
X[:,[1,8]] = temp[:,[1,3]]
print(X[:5,[1,6,8,9]])

Z_perm, L_perm = DMD(X[:,:m], X[:, 1:], N*np.finfo(float).resolution)

L_perm


[[0.50170737 0.50840103 0.50725185 0.50740822]
 [0.5361577  0.52141273 0.5206055  0.52087238]
 [0.52494283 0.52317808 0.52179348 0.52256964]
 [0.53701475 0.53638887 0.53616231 0.53611778]
 [0.52573139 0.5340734  0.53292179 0.53295179]]
[[0.50840103 0.50170737 0.50740822 0.50725185]
 [0.52141273 0.5361577  0.52087238 0.5206055 ]
 [0.52317808 0.52494283 0.52256964 0.52179348]
 [0.53638887 0.53701475 0.53611778 0.53616231]
 [0.5340734  0.52573139 0.53295179 0.53292179]]


array([ 9.99106925e-01+0.j        ,  6.12092220e-01+0.56556659j,
        6.12092220e-01-0.56556659j,  1.00014582e-01+0.77739965j,
        1.00014582e-01-0.77739965j,  4.97408536e-04+0.j        ,
       -4.68393836e-01+0.68215557j, -4.68393836e-01-0.68215557j,
       -7.29778374e-01+0.28419992j, -7.29778374e-01-0.28419992j])

In [ ]:
Ze, Le = DMD_exact(X[:,:m], X[:, 1:], N*np.finfo(float).resolution)
Le
#isto kao L?

array([ 0.99894211+0.j        ,  0.51887989+0.63581299j,
        0.51887989-0.63581299j, -0.05204852+0.69636155j,
       -0.05204852-0.69636155j,  0.02278   +0.j        ,
       -0.11361175+0.j        , -0.67434574+0.j        ,
       -0.51870432+0.56181541j, -0.51870432-0.56181541j])

In [ ]:
Zg, Lg, rg = GEDMD(X[:,:m], X[:,1:], N*np.finfo(float).resolution)
Lg

array([ 0.99894211+0.j        ,  0.51887989+0.63581299j,
        0.51887989-0.63581299j, -0.05204852+0.69636155j,
       -0.05204852-0.69636155j,  0.02278   +0.j        ,
       -0.11361175+0.j        , -0.51870432+0.56181541j,
       -0.51870432-0.56181541j, -0.67434574+0.j        ])

In [ ]:
np.linalg.eigvals(A)[:10]

array([ 0.99895511+0.j        , -0.02983773+0.01680062j,
       -0.02983773-0.01680062j, -0.03220247+0.00853392j,
       -0.03220247-0.00853392j,  0.03447152+0.j        ,
       -0.03115523+0.00941745j, -0.03115523-0.00941745j,
        0.02655775+0.02120484j,  0.02655775-0.02120484j])

In [ ]:
#!pip install pydmd

In [ ]:
import pydmd
dmd1 = pydmd.DMD(svd_rank=10)
dmd1.fit(X[:,:m], X[:,1:])
dmd1.eigs

array([ 0.99894211+0.j        ,  0.51887989+0.63581299j,
        0.51887989-0.63581299j, -0.05204852+0.69636155j,
       -0.05204852-0.69636155j,  0.02278   +0.j        ,
       -0.11361175+0.j        , -0.67434574+0.j        ,
       -0.51870432+0.56181541j, -0.51870432-0.56181541j])

In [ ]:
for i in range(dmd1.eigs.shape[0]):
    print(np.linalg.norm(dmd1.reconstructed_data[:,i]-X[:,i]))

7.115796151869548e-15
2.833224828443359e-14
5.249087432562182e-14
7.673852877147148e-14
1.0240612618128462e-13
1.3128029381943235e-13
1.6220443641266865e-13
1.8585159257912513e-13
2.1239748205499281e-13
2.3746765502419185e-13


In [ ]:
dmd1.reconstructed_data.shape

(300, 10)

In [ ]:
#kako procijeniti s_i?
for i in range(m):
    s=np.zeros((Z.shape[0]))
    for j in range(Z.shape[1]):
        s=s+Z[:,j]*(L[j]**(i))
    print("norma razlike:",np.linalg.norm(s-X[:,i]))
    #print(s,X[:,i], sep="\n")

norma razlike: 8.380852741025778
norma razlike: 7.518788618798513
norma razlike: 7.6044936485452865
norma razlike: 7.489942515084818
norma razlike: 7.41339202255386
norma razlike: 7.465040792299442
norma razlike: 7.460869280515969
norma razlike: 7.361247530479374
norma razlike: 7.379344827998604
norma razlike: 7.359836659952813


In [57]:
#QR za (X Y) kad X i Y imaju "istu jezgru"
N=20; m=7
A = np.random.rand(N,N)
A=A/np.linalg.norm(A)
x = np.ones((N))
X=np.empty((N,m+1))
X[:,0] = x
for i in range(m):
    X[:,i+1]= A @ X[:,i]
#print(X)

Z = np.concatenate((X[:,:m], X[:,1:]), axis=1)
print(Z.shape)

Q, R = np.linalg.qr(Z, mode="reduced")

R_12 = R[:m,m:]
R_22 = R[m:2*m, m:]

Q_1 = Q[:,:m]
Q_2 = Q[:,m:]


print(np.allclose(X[:,1:],Q_1@R_12 + Q_2@R_22))
print(np.allclose(X[:,1:],Q_1@R_12 + Q_2*R_22[:,m-1]))
print(R_22)
print(R.shape)

(20, 14)
True
True
[[ 6.33990211e-17 -1.58355452e-18  6.45301544e-20  2.49024134e-20
   1.35222002e-21 -2.92336628e-22  9.27041910e-08]
 [ 0.00000000e+00  1.05681961e-17  5.97712929e-20  1.21087242e-20
  -1.39312599e-21 -9.44745792e-22 -1.39895522e-07]
 [ 0.00000000e+00  0.00000000e+00 -2.50449034e-19  1.17520300e-20
   2.51145817e-22 -3.16710817e-22  2.83976237e-08]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.58789660e-19
  -5.07302384e-22 -1.36677452e-21 -1.01698255e-07]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -4.53575774e-21  4.11816005e-22 -7.31606894e-08]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.25884059e-21  3.67769204e-08]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.65409449e-07]]
(14, 14)
